In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import FastText, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt

In [2]:
okt = Okt()

/home/jinny/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


## 1. 새로운 input 받기

In [3]:
new_text = '명지니~~~~한창 컨퍼런스 준비중이지 나 일이있어서 컨퍼런스 못가지만 명지니가 발표한번 더해줄거  알아 왱???? 무슨일이야!!!!!!!!!!ㅋㅋㅋㅋㅋㅋ 저는 마스터가......아닙니다.....!!!! 충격고백 햄스터엿던것이다 내가 햄스터란걸 알려주고싶어서 연락햇어... ㅋㅋㅋㅋㄱ 나 컨퍼런스 몬갓ㆍ 앜ㅋㅋㅋㅋㄱ 잊어줘..!.!!!!!! 뒷목을 친다 명지니 기절'

## 2. Tokenization

In [4]:
new_morphs = None
for i in range(len(new_text)):
    new_morphs = okt.morphs(new_text, norm=True, stem=True)

## 3-1. Word2Vec_SG

In [5]:
w2v_sg = Word2Vec.load("word2vec_skip_gram_2.model")

/home/jinny/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
def averaging_w2v_sg(a):
    temp = np.zeros(300)
    cnt = 0
    for token in a:
        try:
            temp += w2v_sg.wv[token]
        except:
            cnt += 1
    return temp/(len(a)-cnt)

In [7]:
i1 = averaging_w2v_sg(new_morphs)

## 3-1. Doc2Vec_PV-DBOW

In [8]:
d2v_dbow = Doc2Vec.load("doc2vec_PV_DBOW.model")

/home/jinny/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
i34 = d2v_dbow.infer_vector(new_morphs)

## 3-1. tf-idf w2v skipgram

In [10]:
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full

with open('zebal_version2.pickle', 'rb') as f:
    df = pickle.load(f)

# 가입 인사 제거
df = df[df['board']!='가입 인사 (등업 필수)']
df = df.reset_index()
df = df.append({'morphs':new_morphs},ignore_index=True)

docs_dict = Dictionary(df['morphs'])
docs_dict.filter_extremes(no_below=20, no_above=0.2)
docs_dict.compactify()

docs_corpus = [docs_dict.doc2bow(doc) for doc in df['morphs']]
model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
docs_tfidf  = model_tfidf[docs_corpus]

docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])

w2v_sg = Word2Vec.load("word2vec_skip_gram_2.model")

tfidf_emb_vecs = np.vstack([w2v_sg.wv[docs_dict[i]] for i in range(len(docs_dict))])

docs_emb = np.dot(docs_vecs, tfidf_emb_vecs)

i2 = docs_emb[-1]

/home/jinny/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## 5. Predict

In [11]:
#Import Classifier

#L1 classifier
with open("gbm_[_w2v_skipgram_avg_]_cv5_L1.pickle", 'rb') as f:
    c1 = pickle.load(f)
    
#L2 classifier
with open("gbm_tfidf_w2v_sg_avg.pickle_2_cv_L2.pickle", 'rb') as f:
    c2 = pickle.load(f)
    
#L3 classifier
with open("gbm_[_doc2vec_PV_DBOW_]_cv2_L3.pickle", 'rb') as f:
    c3 = pickle.load(f)
    
#L4 classifier
with open("gbm_[_doc2vec_PV_DBOW_]_cv3_L4.pickle", 'rb') as f:
    c4 = pickle.load(f)

/home/jinny/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
i1 = pd.DataFrame(i1)
i1 = i1.T
i1.columns = ['w2v_skipgram_avg_{}'.format(str(i)) for i in range(300) ]

i2 = pd.DataFrame(i2)
i2 = i2.T

i34 = pd.DataFrame(i34)
i34 = i34.T
i34.columns = ['doc2vec_PV_DBOW_{}'.format(str(i)) for i in range(300) ]

In [13]:
#make prediction
pred1 = c1.predict(i1)
pred2 = c2.predict(i2)
pred3 = c3.predict(i34)
pred4 = c4.predict(i34)

In [14]:
Output = pred1 + pred2 + pred3 + pred4

In [15]:
Output

array(['ENFP'], dtype=object)